<a href="https://colab.research.google.com/github/Sangh0/DeepLearning-Tutorial/blob/main/current_materials/8_implement_vgg_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG Network 논문 읽고 따라 구현하기

- 이번 챕터에서는 논문을 읽고 따라 구현하는 방법을 배울 거예요
- VGG network 논문은 ImageNet Challenge에서 2위를 차지한 유명한 모델이예요
- 이때 1위는 GoogLeNet이었지만 오히려 VGGNet이 더 주목을 받았는데, 그 이유는 매우 단순한 구조를 가졌다는 점이예요

- 먼저, VGGNet의 구조는 다음과 같아요

<img src = "https://greeksharifa.github.io/public/img/2021-10-24-ImageNet-CNN-models/VGGNet.png">


- 그림으로 표현하면 다음과 같고요

<img src = "https://www.researchgate.net/publication/355049790/figure/fig2/AS:1075420338348033@1633411597542/The-structure-of-VGG16-modelThis-figure-was-created-with-Image-onlineco-and-exported.png">

In [1]:
import torch
import torch.nn as nn
from torchvision.models import vgg16_bn, VGG16_BN_Weights


# Direct Implementation
class VGG16(nn.Module):

    def __init__(self):
        super(VGG16, self).__init__()
        # build VGG16
        self.features = nn.Sequential(
            # first conv block --------------------------------------------------------
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # -------------------------------------------------------------------------

            # second conv block -------------------------------------------------------
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # -------------------------------------------------------------------------

            # third conv block --------------------------------------------------------
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # --------------------------------------------------------------------------

            # fourth conv block --------------------------------------------------------
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # --------------------------------------------------------------------------

            # fifth conv block ---------------------------------------------------------
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # --------------------------------------------------------------------------
        )

        # fully connected layers -------------------------------------------------------
        self.classifier = nn.Sequential(
            nn.Linear(7*7*512, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1000),
        )
        # ------------------------------------------------------------------------------

    def forward(self, x):
        batch_size = x.size(0)
        x = self.features(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x


from torchsummary import summary

summary(VGG16(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

### What is Pre-trained weight?
- 이렇게 VGG16을 직접 구현하고 모델의 파라미터 갯수도 똑같은 걸 확인했어요
- 하지만 직접 구현할 경우 가장 큰 단점은 pre-trained weight가 없다는 것이예요
- 왜 pre-trained weight가 중요한지 비유를 들어드릴게요
- 갓난 아이와 고등학교 수학 과정을 마친 성인이 있다고 가정하죠
- 이 두 사람에게 대학 미적분 과목을 시험을 보게 할 겁니다
- 자, 여기서 우리는 대충 갓난 아이보단 성인이 더 시험을 잘 볼 것이라 예측을 할 수 있죠
- 그 이유는 당연히 초, 중, 고 수학 교육과정을 거치면서 어느 정도 수학의 개념을 배웠다는 것 때문이죠
- 이것을 pre-trained weight를 가지고 있는 모델이라고 생각하시면 됩니다
- 반면, 갓난 아이는 random weight를 가지고 있다고 생각하시면 돼요
- 즉, pre-trained weight로 초기화하여 모델 학습을 하면 random weight로 할 때보다 더 빠르게 학습을 할 수 있고 그 리소스 또한 줄어들 것이라고 예측할 수 있어요

### Pre-trained weight 사용법
- 1. PyTorch에서 제공해주는 기본 모델 사용
    - 가장 보편적인 방법이예요
    - PyTorch에는 유명한 모델들에 한해 구현이 되어 있고 역시 benchmark 데이터셋으로 학습한 pre-trained weight가 존재해요

- 2. 오픈소스에서 제공해주는 모델 사용
    - 이것 역시 위의 방법과 비슷하게 보편적인 방법이예요
    - 다양한 연구원과 개발자들이 오픈소스로 공개한 레퍼지토리를 통해 pre-trained weight를 받을 수 있어요

- 3. 직접 구축
    - 가장 무식하고 시간과 비용이 많이 드는 방법이예요
    - 말 그대로 직접 모델 만들고 직접 사전 학습을 하는 거예요

In [2]:
# pytorch vgg16 로드하기
model = vgg16_bn(weights=VGG16_BN_Weights.IMAGENET1K_V1)
model

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:09<00:00, 59.6MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [4]:
"""
위 코드의 문제점은 마지막 레이어 노드 수가 1,000개인 것을 알 수 있어요
즉, ImageNet dataset의 클래스 수는 1,000개라 맞춰져 있어요

여기서 만약에 내가 가지고 있는 데이터셋의 수가 10개라면 코드를 수정해야 돼요
다음과 같이 수정하면 돼요
"""

class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        model = vgg16_bn(weights=VGG16_BN_Weights.IMAGENET1K_V1)

        self.features = model.features
        self.avgpool = model.avgpool
        self.classifier = model.classifier
        self.classifier[-1] = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

summary(VGG16(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

- 여기까지 VGG16을 구현하는 코드에 대해 살펴봤어요
- 이제 딥러닝 분야 전체적인 overview에 대해서 간단하게 얘기해볼게요

### Overview
**Data Type**
- AI 분야에서는 크게 3개 유형의 데이터셋이 존재해요
- Image, Text, Audio, Tabular
- 이미지, 텍스트, 오디오는 굉장히 복잡도가 높은 편에 속하는 데이터라 주로 비선형 학습에 유리한 딥러닝 모델을 많이 사용해요
- 반면, Tabular는 잘 정리가 되어있고 전처리를 잘만 하면 간단한 머신러닝 알고리즘으로도 잘 학습할 수 있어요

**Vision AI**
- 이미지부터 살펴볼게요
- AI에 이미지 데이터를 적용한 task는 굉장히 많아요
- 크게 Classification, Object Detection, Semantic Segmentation 3개의 task가 존재해요
- Classification은 말 그대로 이미지가 주어지면 해당 이미지의 클래스가 무엇인지 예측하는 작업이예요
- Object Detection은 분류를 넘어서 객체의 위치까지 파악하는 작업이예요

<img src = "https://kajabi-storefronts-production.kajabi-cdn.com/kajabi-storefronts-production/file-uploads/blogs/22606/images/1446e76-f181-6047-4e73-8d8ba3c6a50e_object_detection_1.webp">

- Detection은 위의 사진처럼 사진이 주어지면 사람, 자동차, 신호등, 간판 등을 찾아 box를 그려주고 그 객체의 클래스까지 분류해줘요
- 주로 Faster R-CNN, YOLO 등의 알고리즘이 유명해요

- Semantic Segmentation은 픽셀 단위로 분류를 하는 작업이예요

<img src = "https://miro.medium.com/v2/resize:fit:1400/1*KICInky28yGdU9T45kIL5Q.jpeg">

- 위 이미지처럼 사람, 자동차, 도로, 나무, 건물 등에 해당하는 객체를 픽셀 단위로 분류해줘요

- 이 외에도 Pose Estimation

<img src = "https://miro.medium.com/v2/resize:fit:975/1*DTaPdSzIw4rMmD-6hO15rQ.png">

- Image Generation

<img src = "https://rameenabdal.github.io/StyleFlow/assets/teaser.png">

- 이렇게 Vision AI에 다양한 분야가 존재해요


**Text**
- Natural Language Process 즉, NLP라고도 불리는 task예요
- 불과 GPT가 등장하기 전까지만 해도 NLP는 Question Answering, Text classification, Translation, NER, Text summarization 등의 task가 존재했어요
- 하지만 지금, LLM이 등장한 이후로는 위 세부적인 task들이 필요가 없을 정도로 LLM 하나만으로도 광범위한 task들을 다루기 쉬워졌죠

**Audio**
- Audio도 마찬가지로 Speech Recognition,
Emotion Recognition, Audio Classification 등의 task가 존재해요
